In [1]:
import sys
from collections import defaultdict
import os
import glob

In [2]:
def parse_vcf(vcf_file):
    """
    Parses a VCF file for variants, collects them and return a dictionary
        variants[chrom] = set of (pos,ref,alt) for all PASS variants 
    """
    variants=defaultdict(set)
    with open(vcf_file,'r') as f:
        for line in f:
            if line.startswith("#"):
                continue
            
            #splitting line by tabs
            cols=line.strip().split('\t')

            #vcf columns are arranged as follows: CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO, ...
            chrom = cols[0]
            pos = cols[1]
            id = cols[2]
            ref = cols[3]
            alt = cols[4]
            filt = cols[6]

            if filt == "PASS":
                variants[chrom].add((pos, ref, alt))
    
    return variants


In [ ]:
def compare_vcfs(benchmark_vcf, query_vcf):
    """
    Compare two VCF files (filtered by PASS) in terms of:
        1)Number of variants per chromosome
        2)For each chromosome:
            -calculation of Precision (fraction of query variants that match benchmark variants) OR (true positives/(true positives+false positives))
            -calculation of Recall (fraction of benchmark variants that are matched by query variants) OR (true positives/(true positives + false negatives))
        3)Prints out summary statistics
    """
    #compile statistics for output
    output_lines=[]

    #parsing each vcf file to get variants
    benchmark_variants= parse_vcf(benchmark_vcf)
    query_variants=parse_vcf(query_vcf)

    #get chromosomes of benchmark
    chroms=list(benchmark_variants.keys())

    #summaries (for counting variants across all chromosomes)
    total_variants_benchmark=0
    total_variants_query=0
    total_shared=0

    #comapring variants in each chromosome
    for chrom in chroms:
        variant_set_benchmark=benchmark_variants.get(chrom)
        variant_set_query=query_variants.get(chrom)
        if variant_set_benchmark is None:
            output_lines.append(f"{benchmark_vcf} has no variants for {chrom}")
            continue
        elif variant_set_query is None:
            output_lines.append(f"{query_vcf} has no variants for {chrom}")
            continue


    
        shared=variant_set_benchmark.intersection(variant_set_query)
        
        count1=len(variant_set_benchmark)
        count2=len(variant_set_query)
        shared_count=len(shared)
       
        #updating summary counts
        total_variants_benchmark += count1
        total_variants_query +=count2
        total_shared += shared_count
        output_lines.append(f"For {chrom}, precision is {shared_count/count2} & recall is {shared_count/count1}")

    #Grand summary
    output_lines.append("Overall Summary")
    output_lines.append("--------------")
    output_lines.append(f"The total no. of variants in {benchmark_vcf} is {total_variants_benchmark}")
    output_lines.append(f"The total no. of variants in {query_vcf} is {total_variants_query}")
    output_lines.append(f"The total no. of correct variant found is {total_shared}")
    output_lines.append(f"{round(((total_variants_query-total_shared)/total_variants_query)*100,2)}% of variants called in long-reads Nanopore in this case are false")
    output_lines.append(f"{round((total_shared/total_variants_query)*100,2)}% of variants called in long-reads Nanopore in this case are true")

    return "\n".join(output_lines)

In [4]:
#fethcing the VCF files to work with-- files are in clair3_work
clair3_work_dir=os.path.expanduser("~/clair3_work/run_2_HG005")
query_dir=os.path.expanduser("~/clair3_work/run_2_HG005")
benchmark_vcf=glob.glob(os.path.join(clair3_work_dir,"HG005_benchmark.vcf"))[0]
query_vcf=glob.glob(os.path.join(query_dir,"merge_output.vcf"))[0]
print(benchmark_vcf)
print(query_vcf)

/home/nabil/clair3_work/run_2_HG005/HG005_benchmark.vcf
/home/nabil/clair3_work/run_2_HG005/merge_output.vcf


In [8]:
#processing
with open('HG0005_output.txt','w') as file:
    file.write(compare_vcfs(benchmark_vcf,query_vcf))

For chr1, precision is 0.7846847534139547 & recall is 0.958255830016352
For chr2, precision is 0.8460859942151019 & recall is 0.9602303086135422
For chr3, precision is 0.824118291347207 & recall is 0.961945464175479
For chr4, precision is 0.7939290378150093 & recall is 0.9639248811720534
For chr5, precision is 0.8233861365355454 & recall is 0.9619708840902652
For chr6, precision is 0.7640456126890056 & recall is 0.9617067332708918
For chr7, precision is 0.8094234618018379 & recall is 0.9603810173630438
For chr8, precision is 0.8310696165682442 & recall is 0.9635996296796343
For chr9, precision is 0.7430055651579327 & recall is 0.95967469901686
For chr10, precision is 0.8096021699038142 & recall is 0.9609317728598599
For chr11, precision is 0.8262538964774506 & recall is 0.9645026783064982
For chr12, precision is 0.8142287610983605 & recall is 0.9597596464755335
For chr13, precision is 0.775136591492148 & recall is 0.9632507419881949
For chr14, precision is 0.8233245116915101 & recall i

TypeError: write() argument must be str, not None